# graphic score ideation

- 2 elements to the piece:
  - improvised performance by a musician iterpreting the score, based on accompanying instructions
  - rhythmic/percussive accompaniment by mechanical sounds of drawing arm

- image and drawing arm
  - the score itself should follow certain patterns that the musician will be instructed to iterpret, this is the visual style of the piece. while indeterminate it should be consitent and recognisable accross many iterations
  - this drawing will feature rhythmic patterns (and possibly other, e.g. harmonic, timbral - experiment with drawing arm), these are the percussive accompaniment. they are time based and embedded into the drawing proccess.
  - rhythms possibly based on chord polyrhythms (maybe of a chord built using the hemi-pent scale 13457 or a cluster of the entire hept scale)
  - 1/1 5/4 11/8 3/2 15/8 (factors/multiples of 8)

  8/8:10/8:11/8:12/8:15/8 
  - use divisions of 5 (or mults of 5) to create more interesting relationships
  (or another odd no. or no. that isn't 8)
  - currently in 8th 8/8 notes as base metre
  - in order to be in 5s: 
    - 5/8:8 : 5/10:8 : 5/11:8 : 5/12:8 : 5/15:8
    - so, e.g. 5 beats of 15 against 8  
  - these metres can be used to time rhythmic modes/set patterns to create more interesting interactions

  - ! timing may depend on responsiveness of drawing machine !

- music
  - played on harp
  - tuned to major with a half sharp fourth 1 2 3 ^4 5 6 7 
  - possible focus on 1 3 ^4 5 7 subset
  - improvised using descending glissandi; sweeping motions with a higher probability of being downward rather than upward (this probability can be dynamic)
  - a harp "falling down stairs of light" - concept/name of piece

  - the machine will start drawing and then the musician will start reading it once it has drawn something (machine sould draw fast enought for the musician to play at the correct tempo)
  - the piece will be interpreted live as it is drawn

  - possibly work in motifs into the piece, for the performer to improvise off of, and/or to include in the score

  - similar in stlye to "love's song"? - combining all elements, interlaced at once (noise, free improvisation, motifs, etc)

 -  3rd musical element
  - playing samples of field recordings/ambient non-musical sounds
  - have samples run into a mixer and give the performer instructions to use them
   - parameters of fade-in/out duration, volume, duration of play
   - layer and overlap sounds
   - sound quality/timbre?
  - performer interprets from the score

 - current concept for music
  - 2 drawing machines drawing 2 scores for 2 performers
  - both drawing machines mic'd up (possibly split polyrhythms between them based on 4:3 5:4 relationships, e.g. 15:12 vs. 10:8)
  - performer 1 interprets 1 score to play harp
  - performer 2 interprets the other to play the samples/mixer
  - possible use of effects/techniques such as delay and feedback loops

 - possible 4th musical element:
  - use same operations that generate score and rhythms to generate harmonic/noise/sample based sound via supercollider (audio software)
  - as well as adding another layer, this can enable the piece to be an installation as well as a performance
  - installation version can be reprogrammed to be more ambient and/or have more dynamic range so it varies from slow/ambient to intense/noisy


# graphic score aesthetic / style

- make use of descending/ascending lines to represent glissandi?
  - lines may make rhythmic movements with the machine more reliable by adding "bumps"/kinks in to the lines

- alternatively dots may be used to represent individual notes sounding

- continuously draw over itself, returning to the top of the page, for as long as the piece lasts - so the music can be somewhat continuous

- draw in lines, so the next pass from the top, it can draw in between previous lines
  - without much care for overlap - maybe some intentional (e.g. start second line closer to first with more space to the side/below depending on orientation)

- draw lines verticaly, top-bottom, from right-left (left-right?)
  - draw landscape on paper

- e.g. line may have a lot of horizontal wiggle room and overlap
  - first lines may be centred at x = 10, x += 20, with a wiggle room of 10.
  - the second pass may be drawn at x = 15, x += 20
  - each new pass drawing x += 5 from the start line of the prev pass
  - filling the space between lines gradually from start to end, with intentional overlap built in to each new line

- current idea for final piece: 
  - might look like a series of overlapping downward squiggles, with less upward squiggles overlapping, containing rhythmic patterns and gaps for rests (considering drawing arm speed and movements for readjustment)
  
- can use line tracing code as a basis?

librosa - python library

# references

graphic scores:

- toru takemitsu - corona
- john cage - fontana mix
- cornelius cardew - treatise

- ICA ceatives sound
  - for completely open/less obviously musical style of graphic score

music: 

- my own piece - love's song
  - as a similar style for the sound of the piece

# beginning the code

simplified assumtions to start with:

- drawing speed is at the tempo I want
- drawing speed is the same when the pen is down and when up

basic idea: 

- vertical/diagonal lines drawn downwards (vertical to start?)
- along the full length of a line (~ canvas height), divide the line into nodes signifiying the beats in the polymeters (over the time to draw a line e.g. 10s at 90bpm)
  - 1 beat can be an 8th or 16th (2* or 4* 90bpm) note at in the base rhythm/metre 8
- determine rhythmic patterns (e.g. bounce of an object falling - series of increasingly short durations)
- using chance apply rhythmic patterns along the nodes for each cross-rhythm
- add/subtract where they overlap to create deviations/curves/kinks in the lines
- these kinks should create a rhythmic sounds when the machine draws the score
  - the resulting rhythms may be complex and not very coherent (random sounding)

polyrhythm - 15:12:11:10:8
if it takes 8 seconds to draw a line, the polyrhythm could repeat twice with the base rhythm 8, occuring within 4 second at 2 beats/second
- therefore each line needs an overlapping series of evenly spaced nodes in groups of 16, 20, 22, 24 and 30

the line tracing code can be used as a start point.

In [2]:
from py5canvas import *

In [240]:
import random

create_canvas(900, 600)

no_fill()
background(33,33,22)
stroke(150,200,100,233)
stroke_weight(0.33)

line_spacing = 33 # the distance that the vertical lines are apart

# initial pen position
x, y = 0, 0

def draw_line(sum_lin):
    # Draws a curve connecting a list points [[x1, y1], [x2, y2], ...]
    stroke_weight(0.2)
    begin_shape()
    for i in range(len(sum_lin)):
        curve_vertex(x, sum_lin[i])
    end_shape()

polyrhythm_nodes = [8,10,11,12,15] # 15:12:11:10:8 polyrhythm
line_calcs = []
summed_line = []

def calc_lines(): # separate into 2 functions calc and sum - better modularity?
    global line_calcs, summed_line

    line_calcs = []
    summed_line = []

    for lin in range(int(width/line_spacing)):
        summed_line.append([]) # create new nested array for each line
        
        # calculate y values for each component pulse in the polyrhythm 
        for i in range(len(polyrhythm_nodes)):
            calc_y = 5
            line_calcs.append([])
            for j in range(polyrhythm_nodes[i]):
                line_calcs[i].append(calc_y)
                calc_y += int((height-10)/polyrhythm_nodes[i])
                # print(calc_y)

        # sum calculated line nodes (todo: sum rhythms/patterns expressed via x vals) into 1 line
        for i in range(len(polyrhythm_nodes)):
            for j in range(polyrhythm_nodes[i]):
            # 1/3 chance that a pulse is selected for use per component pulse - random pattern generation
                node_select = random.uniform(0,6) # resolution of polyrhthm
                # print(node_select) 
                if node_select < 1.0: 
                    summed_line[lin].append(line_calcs[i][j])
            if node_select < 1.0: 
                summed_line[lin].append(height-5)   
        summed_line[lin].sort() # not necessary, but good for readability 

# calc_lines()

# print(summed_line)


y_offset = 0
def draw_circles():
    global x, r_val, g_val, alpha_val

    r_val = 170
    g_val = 186
    alpha_val = 155

    for lin in range(int(width/line_spacing)): 

        for j in range(len(summed_line[lin])):
            stroke(r_val, g_val, 80, alpha_val)
            
            draw_line(summed_line[lin])
            
            stroke_weight(1)
            fill(r_val, g_val, 80, alpha_val)
            circle(x, summed_line[lin][j] + y_offset, random.uniform(4,5))
            r_val -= 0.1
            g_val += 0.02
            alpha_val -= 0.167
            # print(r_val, g_val, alpha_val)

        # move to the right
        x += line_spacing

draw_pass = 0
def draw_passes():
    global x, y_offset, draw_pass
    x_init = 11
    x_offset = 0

    x = x_init
    
    while draw_pass <= 11: # higher than division of ~line_spacing to cause overlap
        calc_lines()

        draw_circles()
        
        draw_pass += 1
        x_offset += (line_spacing*1.104)/7 # adding some interesting displacement in overlap
        y_offset += random.uniform(-1, 3)
        x = x_init + x_offset

    
draw_passes()

show()

'''
 - spacing uneven
 - the line needs points at line_length / each polyrhythm_node number

 - create array of node position y vals
 - when drawing, if y == any node pos, draw until the next node pos
  -  I want only nodes of the same cross-rhythm to connect - ones of others will be summed together after
 - in order to draw one line consisting of all these values: 
  - calculate line values for each component rhythm of the polyrhythm
  - sum these values together to get the values for the final line
  - draw with totalled values

 - needed: 
  - calculate_line function
  - line_calc list to store calculated lines
  - sum_line function

 - to draw multiple passes of lines:
  - use modulo to start from the beginning? 
   - or use if: and an x_counter to say if x_counter >= width x = x_init_pos + x_offset
   - then continue to reuse function with x += line_spacing on top of the offset
  - offset x by a small amount for the next row of lines 
  - do this until x_offset >= line_spacing
'''

ValueError: Insufficient points for spline

In [ ]:
# line tracing code - as basis/example

from py5canvas import *

def draw_path(points):
    # Draws a curve connecting a list points [[x1, y1], [x2, y2], ...]
    begin_shape()
    for j in range(len(points)):
        curve_vertex(points[j][0], points[j][1])
    end_shape()

create_canvas(900, 600)

no_fill()
background(0)
stroke(150,200,100,200)
stroke_weight(1)

line_spacing = 4       # the distance that the vertical lines are apart
point_spacing = 11    # the distance between points on a line

num_points = int((height / point_spacing))

# initial pen position
x, y = 1, 0

line_pts = []
line_rec = [[]]

x_vary = [0,0,0,random_uniform(-0.01, 0.025),random_uniform(0.01, 0.067)]
x_vary_subseq = [0,0,0,random_uniform(-0.001, 0.25),random_uniform(0.001, 3)]

# points for the first line

for _ in range(num_points):
    line_rec[0].append([x, y])
    y += point_spacing
    x += random_choice(x_vary)

# draw the first line
draw_path(line_rec[0])

# draw the remaining lines

line_rec_i = 0
x_offset = 0

while x < width:
    # start at the top
    y = 0
   
    x_offset += line_spacing
    
    # create the positions for the new line
    line_pts = []
    for j in range(num_points):
        x = line_rec[line_rec_i][j][0] + x_offset # add current x_offset to prev line contours
        x += random_choice(x_vary_subseq) # add random variation to the new line
        # x values = 0 value contours of prev line + correct offset for new line + random variation
        line_pts.append([x, y]) 
        y += point_spacing

    draw_path(line_pts)

    for j in range(num_points):
        line_pts[j][0] -= x_offset # remove current offset from current line

    line_rec.append(line_pts) # add 0 value line contours to record

    line_rec_i += 1

# show()